In [1]:
from model_loader import ModelLoader
from text_utils import *
import pickle
from scipy import spatial

In [ ]:
# experiment_id, experiment_name
# 01daapwr6w051q9wwqy99jsgfy - Generic
# 01daaqy88qzb19jqz5prjfr76y - Engineering
# 01daaqyn9gbebc92aywnxedp0c - HR
# 01daatanxnrqa35e6004hb7mbn - Marketing
# 01daatbc3ak1qwc5nyc5ahv2xz - Product
# 01dadp74wfv607knpcb6vvxgtg - AI
# 01daayheky5f4e02qvrjptftxv - Ether Engineering

In [5]:
config_path = '../community_detection/data/bert_config.json'
mind_path = '../community_detection/data/se_minds_new.pkl'
model_path = '../community_detection/data/bert_10epc_se_1e-6_sl40.bin'

model_loader = ModelLoader(config_path, model_path)
model = model_loader.loadModel()

In [6]:
#load test files
test_sents = pickle.load(open('../community_detection/data/se_minds_new.pkl','rb'))
test_sents = list(test_sents['sentence'])

In [7]:
print (test_sents)

['Go is extremely well suited for service level tests timeouts async options all have primitives in go', 'Having schemas in the previous step has historically worked for me but I could easily see it moving into this step if it proves to be easier to maintain or reason about', 'Basically what our CI Pipeline will be doing is Restore our packagesTry to build the projectRun our unit testsThe project structure looks like this To configure our Gitlab CI we ll need a', 'In this case you should build three specialised APIs one for each user context', 'This strategy is unnecessarily harsh on smart contracts that churn large quantities of garbage yet have relatively small peak heap sizes and again it may be that in practice a more sophisticated strategy that integrates with the garbage collector is required in order to set quotas to a usefully generic level', 'In order to start the service the deposits service needs some SQS queues created If localstack supports it I think it does through elast

In [8]:
sent_feat_list = []
for sent in test_sents:
    sent_feat_list.append(getBERTFeatures(model, sent, attn_head_idx=-1))

In [15]:
kp_dist_list = []
cs_dist_list = []
text1_list = []
text2_list = []

for i in range(len(test_sents)):
    sent1 = test_sents[i]
    for j in range(i+1,len(test_sents)):
        sent2 = test_sents[j]
        text1_list.append(sent1)
        text2_list.append(sent2)
        tup1 = sent_feat_list[i]
        tup2 = sent_feat_list[j]
        #pos_dist_list.append(getPOSBasedSimilarity_loop(tup1, tup2, sent1, sent2))
        kp_dist_list.append(getKPBasedSimilarity(sent1, sent2, model, tup1, tup2, -3))
        cs_dist_list.append(1-spatial.distance.cosine(tup1[1], tup2[1]))

In [16]:
score_df = pd.DataFrame({'Sent1': text1_list,
                        'Sent2': text2_list,
                        'KP_Similarity': kp_dist_list,
                        'Cos_Similarity': cs_dist_list})

score_df['dist_diff'] = score_df['Cos_Similarity']-score_df['KP_Similarity']
score_df = score_df.sort_values(by='dist_diff',ascending=False)

In [17]:
#score_df[score_df['dist_diff']<0.01]
score_df

,Sent1,Sent2,KP_Similarity,Cos_Similarity,dist_diff
38,In order to start the service the deposits ser...,Here s the implementation of mapSearchPointCha...,0.704467,0.904067,0.199601
24,In this case you should build three specialise...,This strategy is unnecessarily harsh on smart ...,0.618899,0.790762,0.171863
10,Having schemas in the previous step has histor...,In this case you should build three specialise...,0.605962,0.776239,0.170277
21,Basically what our CI Pipeline will be doing i...,So how should REST APIs be built then Build AP...,0.726107,0.891562,0.165455
27,In this case you should build three specialise...,So how should REST APIs be built then Build AP...,0.692904,0.857417,0.164513
29,In this case you should build three specialise...,Here s the implementation of mapSearchPointCha...,0.645458,0.808031,0.162574
33,This strategy is unnecessarily harsh on smart ...,A key aspect of the test stack is that this to...,0.737195,0.890034,0.152839
34,This strategy is unnecessarily harsh on smart ...,Here s the implementation of mapSearchPointCha...,0.745693,0.898407,0.152714
28,In this case you should build three specialise...,A key aspect of the test stack is that this to...,0.624040,0.775698,0.151658
23,Basically what our CI Pipeline will be doing i...,Here s the implementation of mapSearchPointCha...,0.735954,0.875101,0.139147


In [ ]:
key = 21
text1 = score_df['Sent1'][key]
text2 = score_df['Sent2'][key]
layer_idx = 11
kp_sim = getKPBasedSimilarity(text1,text2,model,layer_idx)
cos_sim = getCosineSimilarity(text1, text2,model,layer_idx)